## Check inputs to Ho Chi Minh City regional model before running
Work through all the aspects of the model that don't require it to be actually run before calibrating.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

from summer.utils import ref_times_to_dti

from autumn.tools.inputs.demography.queries import get_population_by_agegroup
from autumn.models.covid_19.detection import create_cdr_function
from autumn.tools.inputs.covid_mmr.queries import get_mmr_testing_numbers
from autumn.tools.utils.utils import apply_moving_average
from autumn.tools.curve.scale_up import scale_up_function
from autumn.tools.project import get_project, TimeSeriesSet
from autumn.settings import Region, Models
from autumn.models.covid_19.constants import AGEGROUP_STRATA, BASE_DATETIME
from autumn.models.covid_19.mixing_matrix.mobility import weight_mobility_data
from autumn.tools.plots.utils import REF_DATE
from autumn.tools.calibration.targets import get_target_series
from autumn.tools import inputs
from autumn.tools.inputs.database import get_input_db
from autumn.tools.utils.display import pretty_print
from autumn.tools.inputs.social_mixing.build_synthetic_matrices import build_synthetic_matrices

In [ ]:
age_integers = [int(group) for group in AGEGROUP_STRATA]
model = Models.COVID_19

## Population

In [ ]:
region = "ho_chi_minh_city"
project = get_project(model, region)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 10))
total_pops = inputs.get_population_by_agegroup(
    AGEGROUP_STRATA, 
    project.param_set.baseline["country"]["iso3"],
    project.param_set.baseline["population"]["region"], 
    year=project.param_set.baseline["population"]["year"]
)
print(f"The estimated total population of HCMC, until 2019, is {round(sum(total_pops) / 1e6, 3)} million.")
ax.bar(age_integers, total_pops, width=4)
ax.set_title("Population of HCMC (2019)", fontsize=20)

## Mobility

In [ ]:
# Collate data together
input_db = get_input_db()
mob_df = input_db.query("mobility", conditions={"iso3": "VNM", "region": "Ho Chi Minh City"})
times = [datetime.strptime(i, "%Y-%m-%d") for i in mob_df["date"]]
google_mob_df = weight_mobility_data(mob_df, project.param_set.baseline["mobility"]["google_mobility_locations"])

# Get plots ready
mob_fig, mob_axes = plt.subplots(1, 2, figsize=(12, 6))
plot_left_date = date(2021, 4, 27)
plot_right_date = times[-1]  # Not sure why this is necessary

# Plot raw mobility data
ax = mob_axes[0]
for mobility_domain in ["grocery_and_pharmacy", "residential", "parks", "retail_and_recreation", "transit_stations"]:
    ax.plot(times, mob_df[mobility_domain], label=mobility_domain)
ax.set_ylim((0., 2.0))
ax.tick_params(axis="x", labelrotation=45)
ax.set_title("Raw Google mobility domains")
ax.legend(loc="lower right")
ax.set_xlim(left=plot_left_date, right=plot_right_date)

# Plot processed mobility data
ax = mob_axes[1]
for location in list(project.param_set.baseline["mobility"]["google_mobility_locations"].keys()):
    ax.plot(times, google_mob_df[location], label=location)
ax.tick_params(axis="x", labelrotation=45)
ax.set_ylim((0., 2.0))
ax.legend(loc="lower left")
ax.set_title("Mobility as implemented in the model")
mob_fig.tight_layout(w_pad=1.5, h_pad=3.5)
ax.set_xlim(left=plot_left_date, right=plot_right_date)

## Mixing matrix
### Confirm mixing matrix details for the region

In [ ]:
print(f"\nFor cluster: {region}")
print(f"\tModelled country: {project.param_set.baseline['country']['iso3']}")
print(f"\tModelled sub-region: {project.param_set.baseline['population']['region']}")
print(f"\tProxy country: {project.param_set.baseline['mixing_matrices']['source_iso3']}")
print(f"\tWhether age adjusted: {project.param_set.baseline['mixing_matrices']['age_adjust']}")

### Display the matrix and the matrix components

In [ ]:
mixing_matrix = build_synthetic_matrices(
    project.param_set.baseline["country"]["iso3"],
    project.param_set.baseline["mixing_matrices"]["source_iso3"],
    AGEGROUP_STRATA,
    project.param_set.baseline["mixing_matrices"]["age_adjust"],
    project.param_set.baseline["population"]["region"]
)

fig = plt.figure(figsize=(12, 8))
positions = [1, 2, 3, 5, 6]
for i_loc, location in zip(positions, mixing_matrix.keys()):
    ax = fig.add_subplot(2, 3, i_loc)
    ax.imshow(
        np.flipud(np.transpose(mixing_matrix[location])), 
        cmap=cm.hot, 
        vmin=0,
        vmax=mixing_matrix[location].max(), 
        origin="lower"
    )
    ax.set_title(location.replace("_", " "))
    ax.set_xticks([])
    ax.set_yticks([])

## Calibration targets

In [ ]:
plot_left_date = date(2021, 4, 29) # first detected case
available_data = ("notifications", "hospital_occupancy", "icu_occupancy", "infection_deaths")

for output in available_data:
    output_fig, output_axes = plt.subplots(1, 1, figsize=(8, 5), sharey="all", sharex="all")
    targets_path = f"../../../autumn/projects/covid_19/vietnam/ho_chi_minh_city/timeseries.json"
    ts_set = TimeSeriesSet.from_file(targets_path)
    output_data = ts_set.get(output)
    output_axes.scatter(ref_times_to_dti(REF_DATE, output_data.times), output_data.values, c="k")
    output_axes.tick_params(axis="x", labelrotation=45)
    output_axes.xaxis.set_major_formatter(mdates.DateFormatter("%d-%m"))
    output_axes.set_xlim(left=plot_left_date)
    output_fig.suptitle(f"Ho Chi Minh City {output}")